# Imports

In [21]:
%reset -f

In [22]:
import io

from pdfminer.high_level import extract_text
import fitz # from pymupdf
import PIL.Image
import tabula

from transformers import pipeline

from tqdm import tqdm
import json

In [23]:
FILENAME1 = 'Data/file1.pdf'
FILENAME2 = 'Data/file2.pdf'

RESULTS1 = 'Results/file1'
RESULTS2 = 'Results/file2'

# Reading contents

## OCR (extract all text from pdf)

In [24]:
text1 = extract_text(FILENAME1)
text1

'BOLETÍN OFICIAL DEL ESTADO\n\nNúm. 27\n\nMartes 1 de febrero de 2022\n\nSupl. J.   Pág. 1\n\nSuplemento del Tablón Edictal Judicial Único\n\nA. Edictos judiciales de carácter general\n\nJuzgados de lo Mercantil\n\nAsturias\n\nGIJÓN. Edicto de 27 de enero de 2022 en procedimiento concursal número 377/2021.\n\nID: J2200301621\n\nÓRGANO JUDICIAL\n\nJuzgado de lo Mercantil Nº 3 de Oviedo\nDirección: Plaza DECANO EDUARDO IBASE S/N\nLocalidad: Gijón\nCódigo Postal: 33207\nProvincia: Asturias\nTeléfono: 985176747\n\nPROCEDIMIENTO\n\nConcursal\nNúmero: 377/2021\nNIG: 3302447120210000336\n\nRESOLUCIÓN PROCESAL\n\nAuto\nFecha: 18/01/2022\nObjeto de la publicación edictal:\n\nNotificación\n\nDESTINATARIOS\n\n"acredores"\n\nSe hace saber que, para conocer íntegramente la resolución o resoluciones a que se \nrefiere el edicto y demás actuaciones del procedimiento indicado, podrá dirigirse a este \nórgano u oficina judicial.\n\nINFORMACIÓN ADICIONAL\n\nE D I C T O\nDª MARIA JOSE ALONSO GONZALEZ, Le

In [25]:
text2 = extract_text(FILENAME2)
text2

'BOLETÍN OFICIAL DEL ESTADO\n\nNúm. 135\n\nMiércoles 7 de junio de 2023\n\nSupl. N.   Pág. 1\n\nSuplemento de Notificaciones\n\nADMINISTRACIÓN LOCAL\n\nMADRID\nAYUNTAMIENTO DE ALCORCÓN\n\nAnuncio de notificación de 2 de junio de 2023 en procedimientos .\n\nID: N2300510206\n\nAYUNTAMIENTO DE ALCORCON RECAUDACION EJECUTIVA MUNICIPAL\nEMPLAZAMIENTO DE NOTIFICACIONES DE PROVIDENCIAS DE APREMIO\n\ntranscurrido  dicho  plazo  no  hubiese  comparecido  el \n\nLos  sujetos  pasivos  que  se  relacionan  figuran  como  deudores  a  este  Ayuntamiento\npor  los  conceptos,  períodos  e  importes  que  seguidamente  se  detallan.  Conforme  a  lo\ndispuesto en el Reglamento General de Recaudación de 29 de Julio de 2.005, por el Sr.\nTesorero Municipal se han dictado, providencias de apremio, por impago de débitos que\na  continuación  se  relacionan,  siendo  tales  providencias  título  suficiente  para  iniciar  el\nprocedimiento de apremio y en caso de no producirse el ingreso en el plazo seña

## Extract tables

In [26]:
dfs1 = tabula.read_pdf(FILENAME1, pages='all')
print(f'Extracted a total of {len(dfs1)} tables.')

Extracted a total of 0 tables.


In [27]:
dfs2 = tabula.read_pdf(FILENAME2, pages='all')
print(f'Extracted a total of {len(dfs2)} tables.')

Extracted a total of 126 tables.


In [28]:
dfs2[0]

,Descripción concepto EJERCICIO,Unnamed: 0,NUMERO DE RECIBO,Unnamed: 1,NIF,IMPORTEPRINCIPAL
0,IMPTO VEHICULOS T. MECANICA 2021,NaN,772815.0,NaN,G82768938,"66,93"
1,IMPTO VEHICULOS T. MECANICA 2020,NaN,138803.0,NaN,B78959525,"66,93"
2,IMPTO VEHICULOS T. MECANICA 2021,NaN,138803.0,NaN,B78959525,"66,93"
3,IMPTO VEHICULOS T. MECANICA 2020,NaN,667237.0,NaN,B84240480,"66,93"
4,IMPTO VEHICULOS T. MECANICA 2020,NaN,305116.0,NaN,B84240480,"175,99"
5,IMPTO VEHICULOS T. MECANICA 2021,NaN,305116.0,NaN,B84240480,"175,99"
6,IMPTO VEHICULOS T. MECANICA 2021,NaN,667237.0,NaN,B84240480,"66,93"
7,IMPTO VEHICULOS T. MECANICA 2020,NaN,368245.0,NaN,B80008550,"66,93"
8,IMPTO VEHICULOS T. MECANICA 2021,NaN,368245.0,NaN,B80008550,"66,93"
9,IMPTO VEHICULOS T. MECANICA 2020,NaN,765490.0,NaN,Y0901902V,"66,93"


# Run dlite

In [29]:
pipe = pipeline("text-generation", model="aisquared/dlite-v2-124m", trust_remote_code=True)
pipe

A new version of the following files was downloaded from https://huggingface.co/aisquared/dlite-v2-124m:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<transformers_modules.aisquared.dlite-v2-124m.bc719f990748ea72be4b6c270df34fc3d37291dc.instruct_pipeline.InstructionTextGenerationPipeline at 0x7f0e3b49eaa0>

In [30]:
pipe('Hello :)')

"Hi everyone. This thread is for the Harry Potter series, although for those of you unfamiliar with the books, I'd be happy to introduce you to a group of common villains that all share a common trait: they all have extremely powerful magical abilities.  Harry Potter and the Sorcerer's Stone, for example, have a pair of powerful abominations, four-headed spiders, a pair of basilisks, and a pair of dwarven galleons. All the villains in this list are extremely dangerous but have the ability to fly."

## On text from `file1`.

In [31]:
def split_string_into_batches(input_string, batch_length):
    batches = []
    for i in range(0, len(input_string), batch_length):
        batch = input_string[i:i+batch_length]
        batches.append(batch)
    return batches


batch_length = 128
result = split_string_into_batches(text1, batch_length)

results1 = {}
for i, batch in tqdm(enumerate(result)):
    results1[f'prompt_{i:003}'] = batch
    results1[f'response_{i:003}'] = pipe(batch)
results1

24it [17:34, 43.93s/it]


{'prompt_000': 'BOLETÍN OFICIAL DEL ESTADO\n\nNúm. 27\n\nMartes 1 de febrero de 2022\n\nSupl. J.   Pág. 1\n\nSuplemento del Tablón Edictal Judicial Únic',
 'response_000': 'BELLEAU de la Semana',
 'prompt_001': 'o\n\nA. Edictos judiciales de carácter general\n\nJuzgados de lo Mercantil\n\nAsturias\n\nGIJÓN. Edicto de 27 de enero de 2022 en proced',
 'response_001': 'A. Edictos judiciales de carácter general\nJuzgados de lo Mercantil\nAsturias\nGIJÓN. Edicto de 27 de enero de 2022 en proced\n\nJuzgados de lo Mercantil\nAsturias\nGIJÓN. Edicto de 27 de energy fructo de 2022 en proced',
 'prompt_002': 'imiento concursal número 377/2021.\n\nID: J2200301621\n\nÓRGANO JUDICIAL\n\nJuzgado de lo Mercantil Nº 3 de Oviedo\nDirección: Plaza DE',
 'response_002': 'ID: J220059914\n\nReferenced: ID: J220059914\n\nThe book Verdión Fernández Sánchez gives us factual information on numerous aspects of the ciencias cartel. Once you have the book, you can review it or give it a watch.',
 'prompt_003': 'C

## On text from `file2`.

In [36]:
result = split_string_into_batches(text2, batch_length)

results2_text = {}
for i, batch in tqdm(enumerate(result)):
    results2_text[f'prompt_{i:003}'] = batch
    results2_text[f'response_{i:003}'] = pipe(batch)
results2_text

85it [1:01:57, 42.28s/it] 

# Save to files

In [32]:
with open(f'{RESULTS1}/results.json', 'w') as f:
    json.dump(results1, f)

with open(f'{RESULTS1}/ocr.txt', 'w') as f:
    f.write(text1)

for idx, df in enumerate(dfs1):
    df.to_csv(f'{RESULTS1}/tables/table_{idx:02}.csv', index_label='Index')

In [ ]:
with open(f'{RESULTS2}/results.json', 'w') as f:
    json.dump(results2_text, f)

# with open(f'{RESULTS2}/ocr.txt', 'w') as f:
#     f.write(text1)

# for idx, df in enumerate(dfs2):
#     df.to_csv(f'{RESULTS2}/tables/table_{idx:02}.csv', index_label='Index')